# Community contribution

Generating a sports brochure - and then in Spanish! Many thanks for the contribution.

In [0]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [0]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [0]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    body: str
    links: List[str]

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [0]:
caps = Website("https://www.nhl.com/capitals/")
print(caps.get_contents())

In [0]:
print(caps.links)

In [0]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the team, \
such as links to an About page, Team, News, Schedule, History, Stats pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "team page", "url": "https://full.url/goes/here/team"},
        {"type": "news page": "url": "https://another.full.url/news"},
        {"type": "schedule page": "url": "https://another.full.url/schedule"},
        {"type": "history page": "url": "https://another.full.url/history"},
        {"type": "stats page": "url": "https://another.full.url/stats"},
        {"type": "standings page": "url": "https://another.full.url/standings"},
    ]
}
"""

In [0]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the team, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, Tickets, Video, Listen, Community, Fans, Youth Hockey, Shop, League, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [0]:
print(get_links_user_prompt(caps))

In [0]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [0]:
get_links("https://www.nhl.com/capitals/")

In [0]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [0]:
print(get_all_details("https://www.nhl.com/capitals/"))

In [0]:
system_prompt = "You are a sports marketing analyst that analyzes the contents of several relevant pages from a sports team website \
and creates a short brochure about the team for prospective fans and players to recruit. Respond in markdown.\
Include details of team history, team culture, team news, and team stats if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [0]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the team in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:40_000] # Truncate if more than 40,000 characters
    return user_prompt

In [0]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [0]:
create_brochure("Washington Capitals", "https://www.nhl.com/capitals")

# Translate to Spanish

In [0]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure in spanish of the team in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:40_000] # Truncate if more than 40,000 characters
    return user_prompt

In [0]:
create_brochure("Washington Capitals", "https://www.nhl.com/capitals")